<a href="https://colab.research.google.com/github/Cipe96/EEG-Recognition/blob/main/Download_Subset_e_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6>**EEG Recognition: Download Subset e Preprocessing**</font>
</br><font size=3>*Marco Cipollina, Riccardo Era*</font>


<p style="font-size:4px;" align="justify">In questo notebook è possibile poter scaricare run specifiche del dataset "EEG Motor Movement/Imagery". È possibile trovare informazioni dettagliate riguardo il dataset originale al seguente <a href="https://physionet.org/content/eegmmidb/1.0.0/">link</a>.</p>
<p style="font-size:4px;" align="justify">Dopo il download della parte interessata è possibile svolgere per una determinata banda il preprocessing reputato migliore per il nostro task.</p>

In [ ]:
%%capture
# evita l' output a video
!pip install mne

In [ ]:
from IPython.display import clear_output
import numpy as np
from google.colab import drive
import mne
import os

In [ ]:
drive.mount('/content/drive', force_remount=True)
! rm -r /content/sample_data

Mounted at /content/drive


Di seguito vengono descritti i task svolti dal paziente per ogni run e la loro durata in minuti:

1.  basale, occhi aperti: 1 min;
2.  basale, occhi chiusi: 1 min;
3.  apre e chiude il pugno destro o sinistro: 2 min;
4.  immagina di aprire e chiudere il pugno destro o sinistro: 2 min;
5.  apre e chiude entrambi i piedi o entrambi i pugni: 2 min;
6.  immagina di chiudere entrambi i piedi o entrambi i pugni: 2 min.

Poi si ripetono gli stessi task descritti al relativo numero per le successive run:

*   Task 3  -->  Run 7 e 11;
*   Task 4  -->  Run 8 e 12;
*   Task 5  -->  Run 9 e 13;
*   Task 6  -->  Run 10 e 14.

In [ ]:
#@title Scegli quale Run scaricare

#@markdown Inserire un valore compreso fra 1 e 14 inclusi:

run = 4 #@param {type:"integer"}
run = f"R{run:02d}"

# Crea la cartella di destinazione se non esiste
dest_dir = f"/content/{run}"
os.makedirs(dest_dir, exist_ok=True)                                            # Crea la cartella se non esiste

In [ ]:
# Da S001 a S109
for i in range(1, 110):
    volontario = f"S{i:03d}"                                                    # Formatta come S001, S002, ..., S109
    url = f"https://www.physionet.org/files/eegmmidb/1.0.0/{volontario}/{volontario}{run}.edf"
    print(f"Scaricando il file {i}/109\n")
    !wget -P {dest_dir} {url}
    clear_output(wait=True)


Scaricando il file 109/109

--2024-11-14 15:38:08--  https://www.physionet.org/files/eegmmidb/1.0.0/S109/S109R04.edf
Resolving www.physionet.org (www.physionet.org)... 18.18.42.54
Connecting to www.physionet.org (www.physionet.org)|18.18.42.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2555616 (2.4M) [application/octet-stream]
Saving to: ‘/content/R04/S109R04.edf’

S109R04.edf         100%[===================>]   2.44M   496KB/s    in 5.1s    

2024-11-14 15:38:13 (486 KB/s) - ‘/content/R04/S109R04.edf’ saved [2555616/2555616]



In [ ]:
files = sorted(os.listdir(dest_dir))

<a name="4"></a>
# **Preprocessing**

Riproduciamo il preprocessing che ci ha portato ai migliori risultati durante la Run01.

Questa fase si divide in tre punti:
*   segmentare i record EEG in campioni di 1,5 secondi;
*   analizzare come le prestazioni cambiano modificando il filtro della banda del segnale;
*   suddividere i campioni in 70% per l'addestramento, 15% per la validazione e il 15% rimanente per il test.



In [ ]:
#@title Parametri segmentazione e suddivisione

duration = 1.5
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

In [ ]:
bande = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 40), 'broadband': (1, 79.9), 'personalizzata': (1, 40), 'ABG': (8,40)}


Salveremo dentro delle apposite cartelle tutti i dataset preprocessati divisi per banda.

La banda che utilizzeremo d'ora in poi è la ABG, che copre le frequenze da 8 a 40 Hz, ossia l'insieme di Alpha, Beta e Gamma.

In [ ]:
#@title Seleziona la banda che vuoi utilizzare:

#@markdown (delta, theta, alpha, beta, gamma, broadband, personalizzata, ABG)

banda = 'ABG' #@param ['delta', 'theta', 'alpha', 'beta', 'gamma', 'broadband', 'personalizzata', 'ABG']

save_path = f'/content/drive/MyDrive/EEG Recognition/Dati preprocessati/{run}/{banda}'

# Verifica se la cartella esiste già
if not os.path.exists(save_path):
    os.makedirs(save_path)                                                      # Crea la cartella
    print(f"Cartella '{save_path}' creata con successo.")
else:
    print(f"La cartella '{save_path}' esiste già.")


Cartella '/content/drive/MyDrive/EEG Recognition/Dati preprocessati/R04/ABG' creata con successo.


Durante la Run01 tutti i volontari sono stati registrati con una frequenza di campionamento pari a 160 Hz.

Qui di seguito abbiamo aggiunto una riga di codice per garantire la frequenza di campionamento sempre a 160. Il volontario 88 di 109, infatti, risulta con frequenza pari a 128 Hz per tutte le run dalla 3 alla 14.

In [ ]:
# Liste per memorizzare i segmenti e le etichette di train, validation e test
all_train_data = []
all_val_data = []
all_test_data = []
all_train_labels = []
all_val_labels = []
all_test_labels = []


# Ciclo su tutti i file .edf
for i, eeg in enumerate(files):
    print(f"Elaborazione del volontario {i + 1} su {len(files)}\n")

    # Carica il file EDF
    raw_data = mne.io.read_raw_edf(f"{dest_dir}/{eeg}", preload=True, verbose = 'CRITICAL')

    #converti a 160 Hz
    if(raw_data.info['sfreq'] != 160):
      raw_data = raw_data.copy().resample(160)
    label = i                                                                   # Ottieni l'etichetta del file

    # Filtro per bande
    raw_data.filter(l_freq=bande[banda][0], h_freq=bande[banda][1], n_jobs=8, verbose = 'CRITICAL')

    # Creazione delle epoche a lunghezza fissa
    epochs = mne.make_fixed_length_epochs(raw_data, duration=duration, overlap=0)

    # Estrazione dei dati come array NumPy
    data = epochs.get_data()                                                    # Dimensioni: (n_epochs, n_channels, n_times)

    # Ora, dopo aver terminato tutte le operazioni di `mne`, si può convertire i dati in float32
    data = data.astype(np.float32)

    data = np.array([epoch.T for epoch in data])                                # Riduci ogni epoca

    clear_output(wait=True)

    # Genera le etichette per ciascuna epoca
    labels_for_file = [label] * len(data)                                       # Assegna la stessa etichetta a tutte le epoche del file

    # Calcolo del numero di segmenti per ciascun set (train, val, test)
    n_samples = len(data)
    n_train = int(n_samples * train_ratio)
    n_val = int(n_samples * val_ratio)

    train_data, val_data, test_data = data[:n_train], data[n_train:n_train + n_val], data[n_train + n_val:]
    train_labels, val_labels, test_labels = labels_for_file[:n_train], labels_for_file[n_train:n_train + n_val], labels_for_file[n_train + n_val:]

    # Aggiungi i segmenti e le etichette di questo file alle liste globali
    all_train_data.append(train_data)
    all_val_data.append(val_data)
    all_test_data.append(test_data)
    all_train_labels.extend(train_labels)
    all_val_labels.extend(val_labels)
    all_test_labels.extend(test_labels)


# Concatena tutti i segmenti e le etichette
train_data = np.concatenate(all_train_data, axis=0)
val_data = np.concatenate(all_val_data, axis=0)
test_data = np.concatenate(all_test_data, axis=0)
train_labels = np.array(all_train_labels)
val_labels = np.array(all_val_labels)
test_labels = np.array(all_test_labels)

# Calcola la media e la deviazione standard globali (usando solo i dati di train per evitare leakage)
media = np.mean(train_data, axis=(0, 1))
dev_std = np.std(train_data, axis=(0, 1))

# Normalizza i dati di train, validation e test usando la media e la deviazione standard
train_data = (train_data - media) / dev_std
val_data = (val_data - media) / dev_std
test_data = (test_data - media) / dev_std


Train set shape: (6235, 240, 64), Train labels shape: (6235,)
Validation set shape: (1308, 240, 64), Validation labels shape: (1308,)
Test set shape: (1417, 240, 64), Test labels shape: (1417,)


In [ ]:
#@title Verifichiamo le dimensioni dei dataset e delle etichette

print(f"Train set shape: {train_data.shape}, Train labels shape: {train_labels.shape}")
print(f"Validation set shape: {val_data.shape}, Validation labels shape: {val_labels.shape}")
print(f"Test set shape: {test_data.shape}, Test labels shape: {test_labels.shape}")

In [ ]:
#@title Salviamo i dataset e le etichette in formato .npy

np.save(f"{save_path}/train_data_{banda}.npy", train_data)
np.save(f"{save_path}/train_labels_{banda}.npy", train_labels)
np.save(f"{save_path}/val_data_{banda}.npy", val_data)
np.save(f"{save_path}/val_labels_{banda}.npy", val_labels)
np.save(f"{save_path}/test_data_{banda}.npy", test_data)
np.save(f"{save_path}/test_labels_{banda}.npy", test_labels)

Una volta salvati i dataset preprocessati di interesse è tutto pronto per poter cambiare notebook e dare tutto in pasto a qualche classificatore.